In [1]:
import projectpath

from importlib import resources
import time
from typing import Iterable, Union

import escher
import ipywidgets as widgets
import bokeh.models
import bokeh.plotting as plt
import jax
import numpy as np
import pandas as pd
import panel as pn

from mosmo.model.core import Molecule, Reaction, Pathway
from mosmo.model.reaction_network import ReactionNetwork
from mosmo.knowledge import kb
from mosmo.sim import fba_gd
import mosmo.preso.escher.pw as pw_files

jax.config.update('jax_enable_x64', True)
prng = jax.random.PRNGKey(int(time.time() * 1000))  # Gets the No GPU warning out of the way

plt.output_notebook()

KB = kb.configure_kb()

Loading BokehJS ...

## TCA cycle - pure cycle, in isolation

In [2]:
tca = KB.find(KB.pathways, 'TCA')[0]
glyox = KB.find(KB.pathways, 'glyoxylate shunt')[0]
network = ReactionNetwork(tca.steps + glyox.steps)

intermediates = [KB(met_id) for met_id in  (
    'cit',
    'acon',
    'icit',
    'akg',
    'succcoa',
    'succ',
    'fum',
    'mal.L',
    'oaa',
    'glx',
)]
boundaries = [met for met in network.reactants if met not in intermediates]

print(f'Network has {network.shape[0]} reactants ({len(intermediates)} intermediates + {len(boundaries)} boundaries) in {network.shape[1]} reactions')
pd.DataFrame(
    network.s_matrix,
    columns=network.reactions.labels(),
    index=network.reactants.labels())

Network has 24 reactants (10 intermediates + 14 boundaries) in 11 reactions


,ICD,AKGDH,SCS,ACN2,FH,CS,SDH,ACN1,MDH,MS,ICL
icit,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
NADP+,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
αkg,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NADPH,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CoA,0.0,-1.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
NAD+,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
NADH,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
succCoA,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATP,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
acCoA = KB('accoa')
akg = KB('akg')
oaa = KB('oaa')

inter2 = [i for i in intermediates if i not in [akg, oaa]]
bnds2 = boundaries + [akg, oaa]
fba = fba_gd.FbaGd(network, inter2, {'mets': fba_gd.ProductionObjective(network, {acCoA: -2, akg: 0, oaa: 0})})
soln = fba.solve()

diag = escher.Builder(
    width=600,
    height=600,
    # map_json=resources.read_text(files.pw, 'glycolysis_ppp_ed.json'),
    map_json=resources.read_text(pw_files, 'central_carbon.json'),
    menu='zoom',
    enable_editing=False,
    never_ask_before_quit=True,
    reaction_data={r: v for r, v in zip(network.reactions.labels(), soln.velocities)},
    reaction_styles = ['color', 'size', 'abs',],
    reaction_scale = [
        {'type': 'value', 'value': 0, 'color': '#eeeeee', 'size': 3},
        {'type': 'value', 'value': 5, 'color': '#1f77b4', 'size': 20},
    ],
)

bflux_plot = plt.figure(
    title = 'Boundary Flux',
    width = 300,
    height = 200,
    tools=[],
    x_range=[boundary.label for boundary in bnds2],
    y_range=(-10, 10),
)
bflux_plot.xaxis.major_label_orientation = np.pi/4
bflux_data = bokeh.models.ColumnDataSource({'met': network.reactants.labels(), 'dmdt': soln.dmdt})
bflux_view = bokeh.models.CDSView(
    source=bflux_data,
    filters=[bokeh.models.IndexFilter([network.reactants.index_of(b) for b in bnds2])]
)
bars = bflux_plot.vbar(
    source = bflux_data,
    view = bflux_view,
    x = 'met',
    top = 'dmdt',
    width=0.5,
    color='#1f77b4',
)

target_accoa = widgets.FloatSlider(
    value=-2,
    min=-10.,
    max=0.,
    step=0.1,
    description='acCoA',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)
target_akg = widgets.FloatSlider(
    value=0,
    min=0,
    max=5.0,
    step=0.1,
    description='αKG',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)
target_oaa = widgets.FloatSlider(
    value=0,
    min=0,
    max=5.0,
    step=0.1,
    description='OAA',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

def update_fba(change):
    fba.update_params({'mets': {acCoA: target_accoa.value, akg: target_akg.value, oaa: target_oaa.value}})
    soln = fba.solve()
    diag.reaction_data = {r: v for r, v in zip(network.reactions.labels(), soln.velocities)}
    bflux_data.data['dmdt'] = soln.dmdt

target_accoa.observe(update_fba, names='value')
target_akg.observe(update_fba, names='value')
target_oaa.observe(update_fba, names='value')

dashboard = widgets.HBox(
    [widgets.VBox([diag], layout = widgets.Layout(width='70%', height='100%')),
     widgets.VBox([
         target_accoa,
         target_akg,
         target_oaa,
         pn.ipywidget(bflux_plot)
     ], layout = widgets.Layout(width='30%', height='100%'))],
    layout = widgets.Layout(width='1000px', height='600px', border='1px solid green')
)
dashboard